In [1]:
import pandas as pd
import bagpy
from bagpy import bagreader
import os
from os import listdir
from os.path import isfile, join
import pathlib as pl
import shutil

# Define paths.

In [6]:
data_dir = pl.Path('../../data/venue_2')
processed_dir = pl.Path('../processed_data/log_tables')

# Create output directories if they do not exist.
for folder in [processed_dir]:
    if not folder.exists():
        folder.mkdir(parents=True)

### Mapping new file locations.

In [3]:
# list of all bag files in data_dir
bag_files = sorted(data_dir.glob('*.bag'))

# renaming table for moving the bag files
temp_map = {}
for f in bag_files:
    index = f.name.split('_')[0]
    new_name = 'justhink20_log_{:02d}.bag'.format(int(index))
    temp_map[f] = processed_dir.joinpath(new_name)
print(temp_map)

for original, temp in temp_map.items():
    shutil.copy(original, temp)

{PosixPath('../../data/venue_2/10_2021-06-11_14-31-27.bag'): PosixPath('../processed_data/log_tables/justhink20_log_10.bag'), PosixPath('../../data/venue_2/1_2021-06-07_08-50-35.bag'): PosixPath('../processed_data/log_tables/justhink20_log_01.bag'), PosixPath('../../data/venue_2/2_2021-06-07_10-23-50.bag'): PosixPath('../processed_data/log_tables/justhink20_log_02.bag'), PosixPath('../../data/venue_2/3_2021-06-07_12-42-01.bag'): PosixPath('../processed_data/log_tables/justhink20_log_03.bag'), PosixPath('../../data/venue_2/4_2021-06-07_14-27-51.bag'): PosixPath('../processed_data/log_tables/justhink20_log_04.bag'), PosixPath('../../data/venue_2/5_2021-06-09_08-41-13.bag'): PosixPath('../processed_data/log_tables/justhink20_log_05.bag'), PosixPath('../../data/venue_2/6_2021-06-09_10-10-24.bag'): PosixPath('../processed_data/log_tables/justhink20_log_06.bag'), PosixPath('../../data/venue_2/7_2021-06-09_12-43-33.bag'): PosixPath('../processed_data/log_tables/justhink20_log_07.bag'), PosixP

# Convert .bag files to csv files.

In [4]:
# converting all bag files to csv files
for temp in temp_map.values():
    name = temp
    print('Reading', str(name))

    b = bagreader(str(name), verbose=True)

    # make a csv file for every topic in the bag
    csvfiles = []
    for t in b.topics:
        data = b.message_by_topic(t)
        csvfiles.append(data)

    completeDf = pd.DataFrame()

    # creating a dataframe for each topic csv
    for index, name in enumerate(csvfiles):
        topic_name = str(name.split('/')[-1].split('.')[0])

        completeDf = completeDf.append(pd.DataFrame(
            [[topic_name], []], columns=["Topic"]))
        dataframe = pd.read_csv(name)
        # reset time
        dataframe['Time'] -= dataframe.iloc[0]['Time']

        l = ['header.stamp.secs', 'header.stamp.nsecs']
        for col in l:
            if col in dataframe.columns:
                dataframe.drop(col, axis=1, inplace=True)

        dataframe.to_csv(name, index=False)

        completeDf = completeDf.append(dataframe)

Reading ../processed_data/log_tables/justhink20_log_10.bag
[INFO]  Data folder ../processed_data/log_tables/justhink20_log_10 already exists. Not creating.
Reading ../processed_data/log_tables/justhink20_log_01.bag
[INFO]  Data folder ../processed_data/log_tables/justhink20_log_01 already exists. Not creating.
Reading ../processed_data/log_tables/justhink20_log_02.bag
[INFO]  Data folder ../processed_data/log_tables/justhink20_log_02 already exists. Not creating.
Reading ../processed_data/log_tables/justhink20_log_03.bag
[INFO]  Data folder ../processed_data/log_tables/justhink20_log_03 already exists. Not creating.
Reading ../processed_data/log_tables/justhink20_log_04.bag
[INFO]  Data folder ../processed_data/log_tables/justhink20_log_04 already exists. Not creating.
Reading ../processed_data/log_tables/justhink20_log_05.bag
[INFO]  Data folder ../processed_data/log_tables/justhink20_log_05 already exists. Not creating.
Reading ../processed_data/log_tables/justhink20_log_06.bag
[INFO

# Clearing temporary bag files.

In [5]:
# remove bag files from log_tables dir
for original, temp in temp_map.items():
    os.remove(str(temp))
    print("Removed temporary bag file {}".format(temp))

Removed temporary bag file ../processed_data/log_tables/justhink20_log_10.bag
Removed temporary bag file ../processed_data/log_tables/justhink20_log_01.bag
Removed temporary bag file ../processed_data/log_tables/justhink20_log_02.bag
Removed temporary bag file ../processed_data/log_tables/justhink20_log_03.bag
Removed temporary bag file ../processed_data/log_tables/justhink20_log_04.bag
Removed temporary bag file ../processed_data/log_tables/justhink20_log_05.bag
Removed temporary bag file ../processed_data/log_tables/justhink20_log_06.bag
Removed temporary bag file ../processed_data/log_tables/justhink20_log_07.bag
Removed temporary bag file ../processed_data/log_tables/justhink20_log_08.bag
Removed temporary bag file ../processed_data/log_tables/justhink20_log_09.bag
